In [ ]:
import pandas as pd
from pathlib import Path

from datasets import load_dataset

from typing import List, Dict

from unique_batches.utils.io import read_token_level_annotated_data

# Load dataset

In [ ]:
dataset = load_dataset("tner/mit_restaurant")

In [ ]:
dataset

In [ ]:
train_df = pd.DataFrame(dataset['train'])
validation_df = pd.DataFrame(dataset['validation'])
test_df = pd.DataFrame(dataset['test'])

df = pd.concat([train_df, validation_df, test_df]).reset_index(drop=True)

In [ ]:
df

# Clean

In [ ]:
def is_only_numeric(tokens: List[str]):
    return all([tok.isdecimal() for tok in tokens])

In [ ]:
df = df[~df['tokens'].apply(is_only_numeric)]

# Put in appropriate form

In [ ]:
label2id = {
    "O": 0,
    "B-Rating": 1,
    "I-Rating": 2,
    "B-Amenity": 3,
    "I-Amenity": 4,
    "B-Location": 5,
    "I-Location": 6,
    "B-RestaurantName": 7,
    "I-RestaurantName": 8,
    "B-Price": 9,
    "B-Hours": 10,
    "I-Hours": 11,
    "B-Dish": 12,
    "I-Dish": 13,
    "B-Cuisine": 14,
    "I-Price": 15,
    "I-Cuisine": 16
}

In [ ]:
id2label = {
    i: label for (label, i) in label2id.items()
}

In [ ]:
def tags_to_labels(tags: List[int], dictionary: Dict[int, str]) -> List[str]:
    labels = list(map(dictionary.get, tags))
    return labels

In [ ]:
df['labels'] = df['tags'].apply(lambda tags: tags_to_labels(tags, id2label))

In [ ]:
def merge_tokens_labels(tokens: List[str], labels: List[str]) -> List[str]:
    token_labels = [f"{token}|{label}" for token, label in zip(tokens, labels)]
    return " ".join(token_labels)

In [ ]:
df['annotation'] = df.apply(lambda row: merge_tokens_labels(row['tokens'], row['labels']), axis=1)

In [ ]:
df

In [ ]:
df['domain'] = 'dummyDomain'
df['intent'] = 'dummyIntent'

df = df[['domain', 'intent', 'annotation']]

df

In [ ]:
datastore = Path("../datastore/").absolute()

In [ ]:
df.to_csv(datastore / "mit_restaurant" / "MITrestaurant.tsv", sep="\t", index=False, header=None)

# Check

In [ ]:
datastore = Path("../datastore/").absolute()

In [ ]:
mit_df = read_token_level_annotated_data(datastore / "mit_restaurant" / "MITrestaurant.tsv")

In [ ]:
mit_df